# Домашнее задание (50 баллов)

В этом домашнем задании вы познакомитесь с основами NLP, научитесь обрабатывать тексты.

В местах, где используется `...` (elipsis), требуется заменить его на код.

Установим необходимые зависимости:

In [2]:
!pip install -U pip
!pip install nltk tqdm seqeval scikit-learn datasets numpy

In [4]:
from typing import List, Dict, Tuple, Callable

## Токенизация (15 баллов)

Токенизация - это процесс преобразования текста в набор токенов.
Наивная реализация разбивает текст по пробелам. Более умные реализации учитывают пунктуацию.

### Библиотека NLTK (2 балла)

Научимся работать с токенизацией NLTK, где уже [реализована](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.word_tokenize) работа с пунктуацией.

https://www.nltk.org/

In [5]:
import nltk

In [6]:
# https://www.nltk.org/nltk_data/
nltk.download("punkt")
nltk.download('punkt_tab')


def tokenize(text: str, language: str = "english") -> List[str]:
    return nltk.tokenize.word_tokenize(text, language)

assert tokenize("") == []
assert tokenize("Hello, world!") == ["Hello", ",", "world", "!"]
assert tokenize("EU rejects German call to boycott British lamb.") == ["EU", "rejects", "German", "call", "to", "boycott", "British", "lamb", "."]

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Нормализация (3 балла)

Добавим нормализацию после токенизации. Пробуем [лемматизацию](https://www.nltk.org/api/nltk.stem.wordnet.html#nltk.stem.wordnet.WordNetLemmatizer) , [стемминг](https://www.nltk.org/api/nltk.stem.snowball.html#nltk.stem.snowball.EnglishStemmer) и [юникод](https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize) нормализацию. Напишем функцию, которая будет принимать на вход токен после токенизации, нормализовать в NFC юникод форму, переводит в нижний регистр, лемматизирует слово и, если слово не изменилось после лемматизации, применяет стемминг.


Создайте функцию `normalize`:
   - Функция `normalize` должна принимать строку `token` и возвращать нормализованный токен.
   - Примените к токену Unicode нормализацию с помощью `unicode_nfc_normalizer`.
   - Преобразуйте токен в нижний регистр.
   - Примените лемматизацию с помощью `lemmatizer`.
   - Если лемматизированный токен отличается от исходного, верните его. В противном случае, примените стемминг с помощью `stemmer` и верните результат.

In [7]:
import unicodedata

nltk.download('wordnet')

from nltk.stem.snowball import EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# питон 3.10 ужасен и в нем nltk 3.2.4 не умеет распаковывать zip архивы!
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [9]:
stemmer = EnglishStemmer()
lemmatizer = WordNetLemmatizer()


def normalize(token: str) -> str:
    """
    Нормализует токен, применяя Unicode нормализацию, преобразование в нижний регистр,
    лемматизацию и стемминг при необходимости.

    :param token: Токен для нормализации
    :return: Нормализованный токен
    """
    normalized_token = unicodedata.normalize('NFC', token)
    lower_case_token = normalized_token.lower()
    lemmatized_token = lemmatizer.lemmatize(lower_case_token)
    if lemmatized_token != lower_case_token:
        return lemmatized_token
    return stemmer.stem(lemmatized_token)


test_tokens = ["Worlds", "churches", "Helping"]
assert [normalize(token) for token in test_tokens] == ["world", "church", "help"]

### Добавляем Словарь (10 баллов)

Современные токенайзеры не только разбивают строки на токены, но и преобразуют последовательность токенов в последовательность числел. Объединим функцию токенизации, нормализации и отображения из токенов в индексы в один объект токенайзера.

Напишите класс `Tokenizer` для токенизации и нормализации текста.

Построение словаря:
   - Создайте метод `_build_vocabulary`, который принимает список текстов `texts` и обновляет словарь токенов.
   - Для каждого текста:
     - Токенизируйте и нормализуйте текст.
     - Обновите счетчик вхождений слов.
   - Для каждого слова, которое встречается не менее `min_count` раз, добавьте слово в словарь `word2idx` и список `idx2word`.

Кодирование и декодирование:
   - Создайте метод `encode_word`, который принимает слово `word` и возвращает его индекс с применением нормализации.
   - Создайте метод `encode`, который принимает текст `text` и возвращает список индексов токенов.
   - Создайте метод `decode`, который принимает список индексов `input_ids` и возвращает текст, вставляя пробелы между токенами.

> Note: для функций, которые могут долго исполнятся (`_build_vocab`), рекомендуется использовать библиотеку tqdm.

In [10]:
from collections import Counter
from tqdm.notebook import tqdm


class Tokenizer:
    def __init__(
            self,
            texts: List[str],
            tokenize_fn: Callable[[str], List[str]] = tokenize,
            normalize_fn: Callable[[str], str] = lambda token: token,
            min_count: int = 1
    ) -> None:
        """
        Инициализация токенизатора.

        :param texts: список текстов для построения словаря
        :param tokenize_fn: функция для токенизации текста
        :param normalize_fn: функция для нормализации токенов
        :param min_count: минимальное количество вхождений слова для включения в словарь
        """
        self.min_count = min_count
        self.tokenize = tokenize_fn
        self.normalize = normalize_fn
        self.word2idx = {"<PAD>": 0, "<BOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.unk_token_id = 3
        self.idx2word = ["<PAD>", "<BOS>", "<EOS>", "<UNK>"]
        self.word2count = Counter()
        self._build_vocabulary(texts)

    def _build_vocabulary(self, texts: List[str]):
        """
        Построение словаря на основе списка текстов.

        :param texts: список текстов
        """
        tokens_list = [self.tokenize(text) for text in tqdm(texts, desc="Tokenizing texts")]
        normalized_tokens = [self.normalize(token) for tokens in tqdm(tokens_list, desc="Normalizing tokens") for token in tokens]
        all_tokens = self.idx2word + normalized_tokens
        self.word2count = Counter(all_tokens)

        for token in self.idx2word:
            self.word2count[token] = self.min_count # чтобы дальше не удалить специальные токены

        index = 0
        self.idx2word = []
        self.word2idx = {}
        for token in tqdm(all_tokens, desc="Building vocabulary"):
            if token not in self.word2idx and self.word2count[token] >= self.min_count:
                self.word2idx[token] = index
                self.idx2word.append(token)
                index += 1


    def encode_word(self, text: str) -> int:
        """
        Кодирование слова в индекс с применением нормализации.

        :param text: слово
        :return: индекс слова
        """
        token = self.normalize(text)
        if token in self.word2idx:
            return self.word2idx[token]
        else:
            return self.word2idx['<UNK>'] # for tf-idf

    def encode(self, text: str) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :return: набор индексов токенов
        """
        tokens = [self.normalize(token) for token in self.tokenize(text)]
        return [self.encode_word(word) for word in tokens]

    def decode(self, input_ids: List[int]) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :return: текст
        """
        return ' '.join([self.idx2word[id] for id in input_ids])

    def __len__(self) -> int:
        """
        Возвращает количество уникальных токенов в словаре.

        :return: количество уникальных токенов
        """
        return len(self.word2idx)

    def __contains__(self, item: str) -> bool:
        """
        Проверяет, содержится ли слово в словаре.

        :param item: слово
        :return: True, если слово содержится в словаре, иначе False
        """
        return item in self.word2idx

    def __str__(self):
        """
        Возвращает строковое представление словаря.

        :return: строковое представление словаря
        """
        return str(self.word2idx)

In [11]:
corpus = ["Hello, world!", "I love Python!"]

tokenizer = Tokenizer(corpus, min_count=1)
encoded = tokenizer.encode("Hello, Python! I love you")
assert tokenizer.decode(encoded) == "Hello , Python ! I love <UNK>"

tokenizer = Tokenizer(corpus, normalize_fn=normalize)
encoded = tokenizer.encode("Hello, Python! I loved you")
assert tokenizer.decode(encoded) == "hello , python ! i love <UNK>"

Tokenizing texts:   0%|          | 0/2 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/2 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/12 [00:00<?, ?it/s]

Tokenizing texts:   0%|          | 0/2 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/2 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/12 [00:00<?, ?it/s]

## TF-IDF (20 баллов)


### Класс TFIDF (10 баллов)

Создайте класс `TFIDF` для вычисления TF-IDF значений. Формулы для подсчёта TF и IDF можно выбрать [тут](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Обучение модели должно осуществляться с помощью метода `fit`, который принимает список строк `docs` и обучает модель на этом корпусе, вызывая метод `add_doc` для каждого документа.

Предсказание TF-IDF значений:
   - Создайте метод `predict`, который принимает список строк `docs` и возвращает матрицу TF-IDF значений.
   - Для каждого документа:
     - Токенизируйте документ.
     - Вычислите TF для каждого термина.
     - Вычислите IDF для каждого термина.
     - Заполните матрицу TF-IDF значений.
   - Нормализуйте строки матрицы, чтобы сумма значений в каждой строке была равна 1.

> Важно! Не забудьте убрать `<UNK>` токен во  время подсчёта TF-IDF

Для функций, которые могут долго исполнятся (`fit`, `predict`), рекомендуется использовать библиотеку tqdm.

In [12]:
from collections import Counter
from typing import List
import numpy as np


class TFIDF:
    def __init__(self, tokenizer: Tokenizer, default_idf = 1.0) -> None:
        """
        Инициализация TFIDF.

        :param tokenizer: токенизатор для преобразования текста в токены
        :param default_idf: значение IDF для неизвестных токенов
        """
        self.tokenizer = tokenizer
        self.num_docs = 0
        self.term2num_docs = [0 for _ in self.tokenizer.word2idx]  # для подсчёта IDF, n_t
        self.default_idf = default_idf

    @property
    def vocab_size(self) -> int:
        """
        Возвращает размер словаря.

        :return: размер словаря
        """
        return len(self.tokenizer)

    def add_doc(self, doc: str) -> None:
        """
        Добавляет документ в модель TFIDF.

        :param doc: документ для добавления
        """
        tokens = set(self.tokenizer.encode(doc))
        if self.tokenizer.unk_token_id in tokens:
            tokens.remove(self.tokenizer.unk_token_id)

        for token in tokens:
            self.term2num_docs[token] += 1
        self.num_docs += 1

    def fit(self, docs: List[str]) -> None:
        """
        Обучает модель TFIDF на корпусе docs.

        :param docs: корпус для обучения
        """
        self.num_docs = 0
        self.term2num_docs = [0 for _ in self.tokenizer.word2idx]   # Сбрасываем счётчики перед обучением

        for doc in tqdm(docs, desc="Training TF-IDF"):
            self.add_doc(doc)

    def predict(self, docs: List[str]) -> np.ndarray:
        """
        Предсказывает TFIDF значения для списка документов.

        :param docs: список документов
        :return: матрица TFIDF значений
        """
        matrix = []
        for doc in tqdm(docs, desc="Computing prediction matrix"):
            token_ids = self.tokenizer.encode(doc)
            counts = Counter(token_ids)
            if self.tokenizer.unk_token_id in counts:
                del counts[self.tokenizer.unk_token_id] # в задании сказано убрать <UNK> токен при подсчете tf-idf
                
            denom = sum(counts.values())
            tf_idf_vector = np.zeros(self.vocab_size)

            for token_id, count in counts.items():
                tf = 0 if denom == 0 else count / denom # если все токены <UNK>, там есть такой assert
                idf = self.idf(token_id)
                tf_idf_vector[token_id] = tf * idf

            if tf_idf_vector.sum() > 0:
                tf_idf_vector /= tf_idf_vector.sum() # нормализация строк
            matrix.append(tf_idf_vector)
        
        return np.array(matrix)
            

    def idf(self, term: int) -> float:
        """
        Вычисляет IDF (обратную частоту документа) для термина.

        :param term: термин
        :return: IDF значение
        """
        # тестовый корпус из ячейки ниже содержит всего 2 текста
        # токен "test" появляется в обоих и в классической формуле
        # idf("test") == log(2 / 2) == 0
        # поэтому в домашнем задании рекомендуется использовать
        # сглаженный вариант подсчёта inverse document frequency smooth:
        # log(N / (n + 1)) + 1
        # https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Inverse_document_frequency
        if self.term2num_docs[term] == 0:
            return self.default_idf # если при обучении tf-idf не встретился такой токен
        n_t = self.term2num_docs[term]
        return np.log(self.num_docs / (1 + n_t)) + 1

Тесты были проверены для такой комбинации формул:

$$
\text{TF} = \frac{f_{t,d}}{\sum_{t' \in d} f_{t',d}}
$$
$$
\text{IDF} = \log{\frac{N}{1 + n_t}} + 1
$$

Если вы выбрали другие формулы для подсчёта, то можно поправить тесты соответственно.

> Можно расширить расширить класс для удобства и поэксперементировать с различными формулами для подсчёта TF и IDF при классификации IMDB датасета ниже.

In [13]:
corpus = ["test test", "not a test"]
tokenizer = Tokenizer(corpus)
tfidf = TFIDF(tokenizer)
tfidf.fit(corpus)

Tokenizing texts:   0%|          | 0/2 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/2 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/9 [00:00<?, ?it/s]

Training TF-IDF:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
assert tfidf.vocab_size == 4 + 3
# 3 токена, один из которых <UNK>
vector = tfidf.predict(["a test string"])[0]
# tf("a") == tf("test") and idf("a") > idf("test")
assert vector[tfidf.tokenizer.word2idx["a"]] > vector[tfidf.tokenizer.word2idx["test"]]

vector = tfidf.predict(["not a test a string"])[0]
assert vector[tfidf.tokenizer.word2idx["a"]] > 2 * vector[tfidf.tokenizer.word2idx["test"]]
assert vector[tfidf.tokenizer.word2idx["a"]] == 2 * vector[tfidf.tokenizer.word2idx["not"]]

assert not np.any(tfidf.predict(["all tokens abscent from vocab should be zeros vector"]))

Computing prediction matrix:   0%|          | 0/1 [00:00<?, ?it/s]

Computing prediction matrix:   0%|          | 0/1 [00:00<?, ?it/s]

Computing prediction matrix:   0%|          | 0/1 [00:00<?, ?it/s]

### Датасет (1 балл)

В качестве датасета вёзмём популярный [набор отзывов на фильмы с сайта IMDB](https://huggingface.co/datasets/stanfordnlp/imdb). Нужно предсказать является ли отзыв позитивным или негативным. Чтобы скачать датасет воспользуемся библиотекой `datasets` из экосистемы `HuggingFace`. Интерфейс датасета похож на словарь, доступ к разным частям осуществляется по названию ключа:

1. Тренировочная часть датасета: `imdb["train"]`
2. Тексты для тренировки: `imdb["train"]["text"]`
3. Лейблы для тренировки: `imdb["train"]["label"]`

In [15]:
from datasets import load_dataset


imdb = load_dataset("imdb")

### Тренируем Токенайзер (2 балла)

Используя `"train"` часть датасета, инициализируйте два токеназйера:
1. Не использующий нормализацию
2. Использующий функцию `normalize`, определённую выше

Сравним размер полученного словаря в обоих случаях.

In [16]:
tokenizer_without_norm = Tokenizer(imdb["train"]["text"])
tokenizer_with_norm = Tokenizer(imdb["train"]["text"], normalize_fn=normalize)

assert len(tokenizer_without_norm) > len(tokenizer_with_norm)

Tokenizing texts:   0%|          | 0/25000 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/25000 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/7056536 [00:00<?, ?it/s]

Tokenizing texts:   0%|          | 0/25000 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/25000 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/7056536 [00:00<?, ?it/s]

### Тренируем TF-IDF Модель (2 балла)

Теперь мы можем натренировать модель на датасете. Обучим две модели, которые будут использовать разные токенайзеры.

In [17]:
tfidf_with_norm = TFIDF(tokenizer_with_norm)
tfidf_with_norm.fit(imdb["train"]["text"])

Training TF-IDF:   0%|          | 0/25000 [00:00<?, ?it/s]

In [18]:
tfidf_without_norm = TFIDF(tokenizer_without_norm)
tfidf_without_norm.fit(imdb["train"]["text"])

Training TF-IDF:   0%|          | 0/25000 [00:00<?, ?it/s]

### Обучим Логистическую Регрессию (5 баллов)

В качестве входов в модель нужно использовать TF-IDF представления документов (`X_train`), в качестве лейблов - 0 и 1, обозначающие нужный класс (`Y_train`). Начнём с модели, которая использует нормализацию.

Используюя тестовый датасет и `logreg.predict` проверьте предсказания модели, вычислив accuracy - количество правильных предсказаний, делённое на количество входных примеров.

In [29]:
from sklearn.linear_model import LogisticRegression

# в imdb["train"] сначала идут 12500 нулей, а затем 12500 единиц
# мне не хватало оперативы в kaggle, поэтому я чуть уменьшил датасет
X_train = tfidf_with_norm.predict(imdb["train"]["text"][10000:15000]) 
Y_train = imdb["train"]["label"][10000:15000] 

Computing prediction matrix:   0%|          | 0/5000 [00:00<?, ?it/s]

In [30]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

LogisticRegression()

In [36]:
# тут тоже чуть уменьшил тестовый сет
X_test = tfidf_with_norm.predict(imdb["test"]["text"][11000:14000])
Y_test = imdb["test"]["label"][11000:14000]

preds = logreg.predict(X_test)
accuracy = (preds == Y_test).sum() / 3000
print("Accuracy: ", accuracy)

Computing prediction matrix:   0%|          | 0/3000 [00:00<?, ?it/s]

Accuracy:  0.7493333333333333


Теперь обучим логистическую регрессию со второй TF-IDF моделью и сравним результаты:

In [37]:
X_train = tfidf_without_norm.predict(imdb["train"]["text"][10000:15000])
Y_train = imdb["train"]["label"][10000:15000]

logreg_without_norm = LogisticRegression()
logreg_without_norm.fit(X_train, Y_train)

X_test = tfidf_without_norm.predict(imdb["test"]["text"][11000:14000])
Y_test = imdb["test"]["label"][11000:14000]

preds = logreg_without_norm.predict(X_test)
accuracy = (preds == Y_test).sum() / 3000
print("Accuracy: ", accuracy)

Computing prediction matrix:   0%|          | 0/5000 [00:00<?, ?it/s]

Computing prediction matrix:   0%|          | 0/3000 [00:00<?, ?it/s]

Accuracy:  0.7366666666666667


In [ ]:
# получилось ожидаемо чуть меньше, но если обучать на всем датасете, то в теории разница будет существеннее

### (Опционально) TfidfVectorizer

Можете изучть класс [TfidfVectorizer](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из библиотеки scikit-learn и сравнить его со своей имплементацией, обучив логистическую регрессию с его помощью.

## $n$-граммные языковые модели (15 баллов)

### Расширяем Токенайзер (3 балла)

Перед созданием языковой модели, расширим токенизационный класс. Добавим два флага в сигнатуру метода `encode`, чтобы управлять добавлением служебных токенов во время токенизации. Существующий метод `decode` уже пропускает `<PAD>` токен, добавим флаг `skip_special_tokens` для пропуска всех специальных токенов.

In [39]:
class BoSTokenizerEoS(Tokenizer):
    def encode(self, text: str, add_bos: bool = True, add_eos: bool = False) -> List[int]:
        """
        Кодирование текста в набор индексов.

        :param text: текст
        :param add_bos: добавление begin-of-sentence токена в начало
        :param add_eos: добавление end-of-sentence токена в конец
        :return: набор индексов токенов
        """
        tokens = ['<BOS>'] if add_bos else []
        tokens += [self.normalize(token) for token in self.tokenize(text)]
        tokens += ['<EOS>'] if add_eos else []
        return [self.encode_word(word) for word in tokens]

    def decode(self, input_ids: List[int], skip_special_tokens: bool = True) -> str:
        """
        Декодирование набора индексов в текст. Вставляет пробел между декодированнми токенами.

        :param input_ids: набор индексов токенов
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: текст
        """
        condition = 4 if skip_special_tokens else 0
        tokens = [self.idx2word[id] for id in input_ids if id >= condition] # не декодирует, если спец. токен
        return ' '.join(tokens)

### Создаём NGram Модель (12 баллов)

Создайте класс `NGramLanguageModel` для построения n-граммной языковой модели. В этом задании вы можете как опираться на предложенную структуру модели, так и сделать свою имплементацию.

Построение модели:
   - Создайте метод `_build_model`, который принимает список текстов `texts` и обновляет частоты n-грамм.
   - Для каждого текста:
     - Токенизируйте текст и добавьте токен `"<EOS>"` в конец.
     - Для каждого токена:
       - Определите префикс длиной `n-1`.
       - Обновите частоты n-грамм и частоты префиксов.

Генерация следующего токена:
   - Создайте метод `generate_next_token`, который принимает префикс `prefix` и возвращает следующий токен.
   - Преобразуйте префикс в кортеж.
   - Получите распределение частот для префикса.
   - Если распределение пустое, верните токен `"<UNK>"`.
   - Верните токен с наибольшей частотой.

Автодополнение текста:
   - Создайте метод `autocomplete`, который принимает текст `text` и максимальную длину `max_len`, и возвращает завершенный текст.
   - Токенизируйте текст.
   - Пока длина токенов меньше `max_len`:
     - Определите префикс длиной `n-1`.
     - Сгенерируйте следующий токен.
     - Добавьте токен в список токенов.
     - Если токен равен `"<EOS>"`, завершите генерацию.
   - Декодируйте и верните текст.

In [40]:
from collections import defaultdict

class NGramLanguageModel:
    def __init__(self, n: int, tokenizer, texts: List[str]):
        """
        Создание n-граммной языковой модели.

        :param n: порядок n-грамм
        :param tokenizer: токенизатор
        :param texts: список текстов
        """
        assert n >= 2
        self.n = n
        self.tokenizer = tokenizer
        self.frequencies = defaultdict(lambda: Counter())  # частота n-грамм
        self.frequencies_of_prefixes = Counter()  # сумма частот n-грамм для префиксов
        self._build_model(texts)

    def _build_model(self, texts: List[str]):
        """
        Построение модели на основе списка текстов.
        Заполнение частот префиксов и следующих за префиксами токенов.

        :param texts: список текстов
        """
        for text in texts:
            tokens = self.tokenizer.encode(text, add_bos=False, add_eos=True)
            for i in range(len(tokens) - self.n + 1):
                prefix = tuple(tokens[i:i + self.n - 1])
                token = tokens[i + self.n - 1]
                self.frequencies[prefix][token] += 1
                self.frequencies_of_prefixes[prefix] += 1

    def generate_next_token(self, prefix: List[int]) -> int:
        """
        Жадная генерация следующего токена по префиксу.

        :param prefix: префикс
        :return: следующий токен
        """
        prefix_tuple = tuple(prefix[-(self.n - 1):]) # на основе последних n - 1 токена
        if prefix_tuple not in self.frequencies:
            return self.tokenizer.unk_token_id
        return max(self.frequencies[prefix_tuple], key=self.frequencies[prefix_tuple].get)

    def autocomplete(self, text: str, max_len: int = 32, skip_special_tokens: bool = True) -> str:
        """
        Автоматическое дополнение текста.

        :param text: текст
        :param max_len: максимальная длина текста
        :param skip_special_tokens: пропуск специальных токенов во время декодирования.
        :return: завершенный текст
        """
        tokens = self.tokenizer.encode(text)
        for _ in range(max_len - len(tokens)):
            prefix = tokens[-(self.n - 1):]
            next_token = self.generate_next_token(prefix)
            tokens.append(next_token)
            if next_token == self.tokenizer.word2idx['<EOS>']:
                break
        return self.tokenizer.decode(tokens, skip_special_tokens=skip_special_tokens)

In [41]:
corpus = ["Hello, world!", "I love Python!", "Hello, Python"]
tokenizer = BoSTokenizerEoS(corpus, min_count=1)

Tokenizing texts:   0%|          | 0/3 [00:00<?, ?it/s]

Normalizing tokens:   0%|          | 0/3 [00:00<?, ?it/s]

Building vocabulary:   0%|          | 0/15 [00:00<?, ?it/s]

In [42]:
ngram_lm = NGramLanguageModel(2, tokenizer, corpus)
assert ngram_lm.autocomplete("Hello, Python", max_len=10) == "Hello , Python !"
assert ngram_lm.autocomplete("Hello, Python", max_len=10, skip_special_tokens=False) == "<BOS> Hello , Python ! <EOS>"

# Комментарии

Если остались вопросы, на которые хочется получить ответ при ревью, это место для них: